In [1]:
import numpy as np
import xsimlab as xs

In [2]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import phydra

In [3]:
def createMultiComp(base_process, comp_label, comp_dim):
    
    @xs.process
    class AddIndexCompLabel(base_process):
        label = xs.variable(intent='out')
        dim = xs.variable(intent='out')
        index = xs.index(dims=comp_label)
        
        def initialize(self):
            self.label = comp_label
            self.dim = comp_dim
            self.index = np.arange(self.dim)
            super(AddIndexCompLabel,self).initialize()
            
    return AddIndexCompLabel

In [4]:
print('xarray-simlab version: ', xs.__version__)
print('phydra version: ', phydra.__version__)

xarray-simlab version:  0.4.1
phydra version:  0+untagged.50.gc1ca6ed.dirty


In [5]:
model = xs.Model({'time':phydra.processes.main.Time, 'grid':phydra.processes.main.Grid0D, 
                  'ibcs':phydra.processes.main.Boundary0D, 'context':phydra.processes.main.GekkoContext, 
                  'C1':createMultiComp(phydra.processes.main.Component,'c1',15), 'C2':createMultiComp(phydra.processes.main.Component,'c2',15),
                  'flux':phydra.processes.main.Flux, 'solver':phydra.processes.main.GekkoSolve})

model

<xsimlab.Model (8 processes, 9 inputs)>
time
    days           [in] ('time',) time in days
grid
    shape          [in]
    length         [in]
ibcs
    ibc            [in] Initial Boundary Conditions
context
C1
    init           [in]
C2
    init           [in]
flux
    growthrate     [in]
    c1_label       [in]
    c2_label       [in]
solver

In [6]:
model.index_vars 

[('time', 'time'), ('C1', 'index'), ('C2', 'index')]

In [7]:
def phydra_setup(model, input_vars, output_vars):
    return xs.create_setup(model=model, 
                           # necessary for xsimlab
                           clocks={'clock': [0, 1]},
                           input_vars=input_vars,
                           output_vars=output_vars)

in_ds = phydra_setup(
    model=model,
    input_vars={
        'time__days': ('time', np.arange(0, 15)),
        'C1':{'init':1}, 
        'C2':{'init':2}, #the way dims are handled need to be specified in each flux!
        
        'flux':{'growthrate':0.1,
                'c1_label':'c1',
                'c2_label':'c2'}
    },
    output_vars={
        'solver__output1':None
    }
                 )

In [8]:
with model:
    out_ds = in_ds.xsimlab.run()

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Gekko math dict stores: dict_items([('c1', array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=object))])
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Gekko math dict stores: dict_items([('c1', array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=object)), ('c2', array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=object))])
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
[[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]]
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
SolveInit
finalize
[('c1', 15), ('c2', 15)]
XXX
filled_output
[[1.         1.11111111 1.2345679  1.37174211 1.5241579  1.69350878
  1.88167642 2.09075158 2.32305731 2.58117479 2.86797199 3.18663555
  3.54070616 3.93411796 4.37124217]
 [1.         1.11111111 1.2345679  1.37174211 1.5241579  1.69350878
  1.88167642 2.09075158 2.32

In [9]:
out_ds

<xarray.Dataset>
Dimensions:           (c1: 15, clock: 2, time: 15)
Coordinates:
  * clock             (clock) int64 0 1
  * time              (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
Dimensions without coordinates: c1
Data variables:
    C1__init          int64 1
    C2__init          int64 2
    flux__c1_label    <U2 'c1'
    flux__c2_label    <U2 'c2'
    flux__growthrate  float64 0.1
    grid__length      int64 1
    grid__shape       int64 1
    ibcs__ibc         float64 nan
    index             (c1) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
    solver__output1   (c1, time) float64 1.0 1.111 1.235 ... 3.541 3.934 4.371
    time__days        (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14

In [10]:
out_ds.solver__output1.plot(x='time')
